In [1]:
import numpy as np
from numpy import *
import matplotlib
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
import requests
import pickle
import re
import itertools
import functools
import collections
import string
import time
from bs4 import BeautifulSoup
from sortedcontainers import SortedList # http://www.grantjenks.com/docs/sortedcontainers/sortedlist.html#sortedlist

from adventlib.common import *
# n, m, a = lines_as_matrix_nm(lines)
# a = lines_as_matrix(lines)
# a = lines_as_digit_matrix(lines)
# chunks = split_iter_by_start_indices(a, indices)
# chunks = split_iter_by_bools(a, bools)

# from adventlib.point_2d import *
# from adventlib.point_3d import *

YEAR = 2022
DAY = int('16')

submit1, submit2 = generate_submits(YEAR, DAY)

while True:
  try:
    raw_input = get_input(YEAR, DAY)
    break
  except Exception as e:
    print(e)
    time.sleep(1)

In [15]:
lines, groups = linesplit("""
Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II
""".strip())

In [21]:
lines, groups = linesplit(raw_input, verbose=True)

64 lines
1 groups
>>>
Valve MU has flow rate=0; tunnels lead to valves VT, LA
Valve TQ has flow rate=0; tunnels lead to valves HU, SU
Valve YH has flow rate=0; tunnels lead to valves CN, BN
...


In [22]:
names = set()
rates = {}
neighs = {}

for line in lines:
  if m := re.findall(r'^Valve (.*) has flow rate=(.*); tunnels? leads? to valves? (.*)$', line):
    m = m[0]
    name = m[0]
    rate = int(m[1])
    neigh = m[2].split(', ')
    
    names.add(name)
    rates[name] = rate
    neighs[name] = neigh
  else:
    raise Exception()

In [23]:
paths = {}

for start in names:
  visited = set()
  distance = {start: 0}
  q = collections.deque([start])
  prev = {}
  
  while q:
    t = q.popleft()
    
    for tt in neighs[t]:
      if tt not in visited:
        visited.add(tt)
        q.append(tt)
        distance[tt] = distance[t] + 1
        prev[tt] = t
  
  for name in names:
    t = name
    path = []
    
    while t != start:
      path.append(t)
      t = prev[t]
    
    paths[(start, name)] = path[::-1]
    
byrate = sorted(names, key=lambda x: rates[x])
byrate

['SF',
 'HU',
 'YN',
 'VH',
 'IL',
 'NJ',
 'IK',
 'MU',
 'HF',
 'TZ',
 'OJ',
 'IP',
 'WX',
 'BN',
 'RJ',
 'NR',
 'AA',
 'IX',
 'YI',
 'TA',
 'UU',
 'KA',
 'EO',
 'FM',
 'AN',
 'AM',
 'TQ',
 'ML',
 'EE',
 'PE',
 'XG',
 'XD',
 'MH',
 'HG',
 'LR',
 'JC',
 'HN',
 'YH',
 'OL',
 'LA',
 'XZ',
 'LY',
 'OM',
 'PX',
 'KO',
 'GQ',
 'BB',
 'UJ',
 'CV',
 'RP',
 'RI',
 'VT',
 'JQ',
 'CN',
 'HR',
 'GG',
 'KV',
 'LI',
 'DT',
 'QF',
 'SU',
 'YA',
 'NS',
 'UH']

In [24]:
names

{'AA',
 'AM',
 'AN',
 'BB',
 'BN',
 'CN',
 'CV',
 'DT',
 'EE',
 'EO',
 'FM',
 'GG',
 'GQ',
 'HF',
 'HG',
 'HN',
 'HR',
 'HU',
 'IK',
 'IL',
 'IP',
 'IX',
 'JC',
 'JQ',
 'KA',
 'KO',
 'KV',
 'LA',
 'LI',
 'LR',
 'LY',
 'MH',
 'ML',
 'MU',
 'NJ',
 'NR',
 'NS',
 'OJ',
 'OL',
 'OM',
 'PE',
 'PX',
 'QF',
 'RI',
 'RJ',
 'RP',
 'SF',
 'SU',
 'TA',
 'TQ',
 'TZ',
 'UH',
 'UJ',
 'UU',
 'VH',
 'VT',
 'WX',
 'XD',
 'XG',
 'XZ',
 'YA',
 'YH',
 'YI',
 'YN'}

In [78]:
best = 0

for order in tqdm(itertools.permutations(byrate[-10:])):
  cur = 'AA'
  t = 30
  score = 0
  
  for i in order:
    t -= len(paths[(cur, i)])
    t -= 1
    if t <= 0:
      break
    score += rates[i] * t
    cur = i
  
  if score > best:
    # print(order)
    best = score
    print(best)

0it [00:00, ?it/s]

625
722
734
760
781
854
875
925
953
977
1010
1027
1043
1046
1055
1063
1075
1094
1121
1137
1140
1149
1154
1195
1199
1207
1219
1247
1312
1316
1328
1347
1361
1366
1377
1383
1431
1438
1487
1531
1571
1613
1653


In [66]:
submit1(1653)

Submitting 1653...


True

In [43]:
best = 0

for order in tqdm(itertools.permutations(byrate[-13:])):
  cur1 = 'AA'
  cur2 = 'AA'
  score = 0
  
  t = 26
  for i in order[::2]:
    t -= len(paths[(cur1, i)])
    t -= 1
    if t <= 0:
      break
    score += rates[i] * t
    cur1 = i
  
  t = 26
  for i in order[1::2]:
    t -= len(paths[(cur2, i)])
    t -= 1
    if t <= 0:
      break
    score += rates[i] * t
    cur2 = i
  
  if score > best:
    # print(order)
    best = score
    print(best)

0it [00:00, ?it/s]

797
805
845
859
866
898
899
906
955
975
976
987
1050
1063
1071
1073
1084
1089
1095
1102
1106
1110
1139
1212
1218
1228
1233
1242
1246
1247
1258
1320
1333
1341
1354
1355
1360
1363
1376
1381
1387
1388
1390
1396
1409
1414
1415
1417
1430
1435
1444
1446
1449
1455
1456
1460
1464
1474
1477
1481
1482
1485
1486
1496
1499
1503
1506
1513
1527
1549
1551
1552
1555
1557
1558
1567
1570
1588
1589
1601
1615
1625
1626
1628
1631
1636
1637
1640
1643
1646
1647
1655
1660
1662
1664
1670
1671


KeyboardInterrupt: 

In [ ]:
submit2(best)

In [11]:
import multiprocessing

In [21]:
best = 0

def proc_order(order):
  cur1 = 'AA'
  cur2 = 'AA'
  score = 0
  
  t = 26
  for i in order[::2]:
    t -= len(paths[(cur1, i)]) + 1
    if t <= 0:
      break
    score += rates[i] * t
    cur1 = i
  
  t = 26
  for i in order[1::2]:
    t -= len(paths[(cur2, i)]) + 1
    if t <= 0:
      break
    score += rates[i] * t
    cur2 = i
  
  return score

In [33]:
[(i, rates[i]) for i in byrate[-15:]]

[('RP', 3),
 ('RI', 4),
 ('VT', 5),
 ('JQ', 6),
 ('CN', 7),
 ('HR', 11),
 ('GG', 12),
 ('KV', 13),
 ('LI', 15),
 ('DT', 16),
 ('QF', 17),
 ('SU', 19),
 ('YA', 21),
 ('NS', 23),
 ('UH', 25)]

In [26]:
N = 14
M = 6

a = byrate[-N:]
best = 0

for i in tqdm(range(1 << N)):
  a1 = []
  a2 = []
  
  for j in range(N):
    if i & (1 << j) > 0:
      a1.append(a[j])
    else:
      a2.append(a[j])
      
  if len(a1) <= M or len(a2) <= M or len(a1) < len(a2):
    continue
  
  best1 = 0
  best2 = 0
  
  for order in itertools.permutations(a1):
    cur1 = 'AA'
    score = 0

    t = 26
    for i in order:
      t -= len(paths[(cur1, i)]) + 1
      if t <= 0:
        break
      score += rates[i] * t
      cur1 = i
    
    if score > best1:
      best1 = score
  
  for order in itertools.permutations(a2):
    cur1 = 'AA'
    score = 0

    t = 26
    for i in order:
      t -= len(paths[(cur1, i)]) + 1
      if t <= 0:
        break
      score += rates[i] * t
      cur1 = i
    
    if score > best2:
      best2 = score
  
  if best1 + best2 > best:
    best = best1 + best2
    print(best, a1, a2)

  0%|          | 0/16384 [00:00<?, ?it/s]

1669 ['RI', 'VT', 'JQ', 'CN', 'HR', 'GG', 'KV'] ['LI', 'DT', 'QF', 'SU', 'YA', 'NS', 'UH']
1757 ['RI', 'VT', 'JQ', 'CN', 'HR', 'GG', 'LI'] ['KV', 'DT', 'QF', 'SU', 'YA', 'NS', 'UH']
1828 ['RI', 'VT', 'JQ', 'CN', 'HR', 'GG', 'DT'] ['KV', 'LI', 'QF', 'SU', 'YA', 'NS', 'UH']
1981 ['RI', 'VT', 'JQ', 'CN', 'GG', 'LI', 'DT'] ['HR', 'KV', 'QF', 'SU', 'YA', 'NS', 'UH']
1995 ['RI', 'VT', 'JQ', 'CN', 'HR', 'GG', 'SU'] ['KV', 'LI', 'DT', 'QF', 'YA', 'NS', 'UH']
2023 ['RI', 'VT', 'JQ', 'CN', 'GG', 'LI', 'SU'] ['HR', 'KV', 'DT', 'QF', 'YA', 'NS', 'UH']
2098 ['RI', 'VT', 'JQ', 'CN', 'GG', 'DT', 'SU'] ['HR', 'KV', 'LI', 'QF', 'YA', 'NS', 'UH']
2121 ['RI', 'VT', 'JQ', 'GG', 'LI', 'DT', 'SU'] ['CN', 'HR', 'KV', 'QF', 'YA', 'NS', 'UH']
2136 ['RI', 'JQ', 'CN', 'GG', 'LI', 'DT', 'SU'] ['VT', 'HR', 'KV', 'QF', 'YA', 'NS', 'UH']
2145 ['JQ', 'CN', 'GG', 'LI', 'DT', 'QF', 'SU'] ['RI', 'VT', 'HR', 'KV', 'YA', 'NS', 'UH']
2182 ['RI', 'VT', 'JQ', 'HR', 'KV', 'YA', 'NS'] ['CN', 'GG', 'LI', 'DT', 'QF', 'SU', 'UH']

In [52]:
submit2(best)

Submitting 2210...
[<p>That's not the right answer.  If you're stuck, make sure you're using the full input data; there are also some general tips on the <a href="/2022/about">about page</a>, or you can ask for hints on the <a href="https://www.reddit.com/r/adventofcode/" target="_blank">subreddit</a>.  Because you have guessed incorrectly 5 times on this puzzle, please wait 5 minutes before trying again. (You guessed <span style="white-space:nowrap;"><code>2210</code>.)</span> <a href="/2022/day/16">[Return to Day 16]</a></p>]


/Users/abra/projects/jupyter/adventofcode2022/adventlib/common.py:51: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 51 of the file /Users/abra/projects/jupyter/adventofcode2022/adventlib/common.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  print(BeautifulSoup(response.text).select('main>article>p'))


AssertionError: 